# 🚀 Building a Startup Advisor System Using Strands Agents: Step-by-Step Walkthrough


**System Architecture:**
```
Basic Agent → Research Agent → Agent Swarm → Specialized Teams → Complete System
```

Let's get started! 


## Setup and Configuration

First, let's import our libraries and set up the foundation for our agents.


In [ ]:
import os
import textwrap
from dotenv import load_dotenv
from strands import Agent
from strands.models import BedrockModel

load_dotenv()

# Set environment variables for container runtime
os.environ['PATH'] = f"/usr/local/bin:{os.environ['PATH']}"
os.environ['MCP_CONTAINER_RUNTIME'] = 'finch'

In [7]:
# Configure a Bedrock Model
model = BedrockModel(
    model_id="us.amazon.nova-micro-v1:0",
    region_name="us-west-2"
)

print("LLM: Amazon Nova Micro")
print("Region: us-west-2")


LLM: Amazon Nova Micro
Region: us-west-2


In [8]:
# Utility function for consistent, readable output
DISPLAY_WIDTH = 120

def print_response(text, title=None, width=DISPLAY_WIDTH):
    """Display agent responses in a clean, formatted way"""
    if title:
        print(f"\n🤖 {title}")
        print("─" * width)
    
    # Wrap text to fit our display width
    wrapped = textwrap.fill(str(text), width=width)
    print(wrapped)
    
    if title:
        print("─" * width)

def print_step(step_num, title):
    """Print step headers consistently"""
    print(f"\n{'='*60}")
    print(f"🎯 STEP {step_num}: {title}")
    print(f"{'='*60}")



---

## Step 1: Basic Startup Advisor

Let's start with the simplest possible agent - one that can give basic startup advice without any external tools. 


<img src="architecture/step1.png" alt="Description" width="500">


In [9]:
print_step(1, "Basic Startup Advisor")

# Create our first agent with a specific role and personality
basic_advisor = Agent(
    system_prompt="""You are an experienced startup advisor with 10+ years helping entrepreneurs. 
    Provide practical, actionable advice that's encouraging but realistic. 
    Keep responses concise and focus on the most important points.""",
    model=model
)

# Test our basic advisor
startup_idea = "A mobile app that helps people find and share local hiking trails"

print(f"Startup Idea: {startup_idea}")
print("\nAsking for initial advice...")

response = basic_advisor(f"Give me your initial thoughts on this startup idea: {startup_idea}");

print_response(response, "Basic Advisor Response")



🎯 STEP 1: Basic Startup Advisor
Startup Idea: A mobile app that helps people find and share local hiking trails

Asking for initial advice...
Your idea for a mobile app to help people find and share local hiking trails is promising and has a lot of potential. Here are some key points to consider:

1. **Market Demand**: There's a growing interest in outdoor activities, especially hiking. Your app could tap into this trend by providing a platform that caters to both casual hikers and enthusiasts.

2. **Unique Selling Proposition (USP)**: To stand out, focus on what makes your app unique. This could be detailed trail maps, user-generated content, community features, or integration with fitness tracking.

3. **User Experience (UX)**: Ensure the app is intuitive and user-friendly. A clean, easy-to-navigate interface will encourage more users to explore and share trails.

4. **Local Community**: Leverage local partnerships to get the app noticed. Collaborate with local hiking clubs, parks, 

---

## Step 2: Research-Powered Agent

Now let's upgrade our agent with web search capabilities using **MCP (Model Context Protocol)**. 

<img src="architecture/step2.png" alt="Description" width="500">


In [10]:
print_step(2, "Research-Powered Agent with Web Search")

# Import MCP libraries for connecting to external services
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Check if we have the required API key
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

if PERPLEXITY_API_KEY:
    # print("✅ Perplexity API key found")
    
    # Create connection to Perplexity search service via Docker
    search_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="finch",
                args=[
                    "run", "-i", "--rm", 
                    "-e", "PERPLEXITY_API_KEY",
                    "mcp/perplexity-ask"
                ],
                env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY}
            )
        )
    )
    
else:
    print("No Perplexity API key found")
    search_client = None



🎯 STEP 2: Research-Powered Agent with Web Search


In [11]:
# Function to test our research agent
def test_research_agent():
    """Test the research agent with web search capabilities"""
    
    
    with search_client:
        # Get available search tools
        search_tools = search_client.list_tools_sync()
        
        print(f"🛠️ Available tools: {[tool.tool_name for tool in search_tools]}")
        
        # Create research agent with search capabilities
        research_agent = Agent(
            system_prompt="""You are a market researcher who finds current, factual information.
            Use web search to gather data about markets, competitors, and trends.
            Provide specific data points and cite your sources. Keep responses focused and informative.""",
            model=model,
            tools=search_tools  # This gives the agent search capabilities!
        )
        
        # Test the research capabilities
        query = "hiking app market size and main competitors 2024"
        print(f"Research Query: {query}")
        
        result = research_agent(query);
        
        print_response(result, "Research Agent Results")
        
        return result

# Test our enhanced agent
research_result = test_research_agent()


🛠️ Available tools: ['perplexity_ask', 'perplexity_research', 'perplexity_reason']
Research Query: hiking app market size and main competitors 2024
<thinking> To provide the most current and accurate information about the hiking app market size and main competitors in 2024, I will need to perform a detailed market research using the available tools. I should first gather the latest market size data and then identify the main competitors in the hiking app market. After gathering the necessary data, I will compile the information and present it to the user.</thinking>


Tool #2: perplexity_research

Tool #3: perplexity_research
Here's the synthesized information about the hiking app market size and main competitors in 2024 based on the latest available data:

### Hiking App Market Size in 2024

The hiking app market size has been reported with significant variation across different market research firms. Verified Market Research and Dev Technosys both estimate the market at approximately

---

## Step 3: Multi-Agent Collaboration with Swarm

Now we'll introduce **agent swarms** - a powerful way to coordinate multiple AI agents working together. Think of it like having a team of specialists:

- **Lead Agent**: Coordinates the team and delegates tasks
- **Specialist Agents**: Created on-demand for specific research areas

<img src="architecture/step3.png" alt="Description" width="500">


In [12]:
print_step(3, "Multi-Agent Collaboration with Swarm")

# Import the swarm tool for agent coordination
from strands_tools import swarm


def test_swarm_collaboration():
    """Test coordinating multiple agents with swarm"""
    
    # Create coordinator with both search and swarm tools
    with search_client:
        search_tools = search_client.list_tools_sync()
        
        team_coordinator = Agent(
            system_prompt="""You are a Research Team Lead who coordinates market research.
            Use the swarm tool to create specialist agents for different research areas.
            Each specialist agent can research their assigned topic.
            Coordinate the team to provide comprehensive market analysis.""",
            model=model,
            tools=search_tools + [swarm]  # Search tools + coordination
        )
        
        tools_available = f"web search + swarm coordination"
    
    print(f"👥 Team Coordinator created with: {tools_available}")
    
    # Give the coordinator a complex task requiring multiple specialists
    complex_task = """
    Research the outdoor recreation app market for a hiking trail app startup.
    
    Create a swarm of 2 specialist agents to research:
    1. Market size, growth trends, and user demographics
    2. Key competitors, their features, and market positioning
    
    Coordinate the research and provide a comprehensive summary.
    """
    
    print("📋 Assigning complex research task to team coordinator...")
    
    team_result = team_coordinator(complex_task);
    
    print_response(team_result, "Team Coordination Results")
    
    return team_result

# Test multi-agent collaboration
swarm_result = test_swarm_collaboration()

print("\n✅ Step 3 Complete: Multiple agents can now work together as a coordinated team!")



🎯 STEP 3: Multi-Agent Collaboration with Swarm
👥 Team Coordinator created with: web search + swarm coordination
📋 Assigning complex research task to team coordinator...
<thinking> To conduct comprehensive market research for a hiking trail app startup, I will create a swarm of 2 specialist agents, each focusing on a different research area. The first agent will research market size, growth trends, and user demographics, while the second will focus on key competitors, their features, and market positioning. After both agents gather the necessary information, I will coordinate their findings to provide a comprehensive summary.</thinking>


Tool #4: swarm

Tool #5: swarm
## Outdoor Recreation App Market Research Outdoor# Recreation App Market Research# Outdoor Recreation App Market Research

## Market

As the Research Team Lead,

As the first contributor in this phase, Outdoor Recreation App Market Analysis - Initial Overview
The outdoor recreation app market is experiencing I'll provide

# Building on Market Research: Engaging Users & Differentiating in the Hiking App Market

I'll build upon our previous research by exploring opportunities for user engagement and differentiation strategies:

## User Engagement & Retention Insights
- **Community engagement metrics**: Apps with strong social features show 30% higher retention rates 
- **Content engagement patterns**: User-generated content (trail reviews, photos) drives 60% more return sessions
- **Feature adoption timeline**: New users prioritize navigation first, then tracking, and finally social features
- **Session frequency**: Average active hiking app user engages 2-3 times weekly during season, drops 70% off-season

## Potential Differentiation Opportunities
- **Hyperlocal expertise**: Market gap for apps with deep, specialized knowledge of specific trail systems
- **AR trail visualization**: Only 8% of hiking apps leverage augmented reality for trail visualization
- **Ecosystem integration**: Partnership opportun

<thinking> The research conducted by the specialist agents provides a comprehensive overview of the outdoor recreation app market, particularly focusing on hiking trail apps. Here is a consolidated summary of the findings:

**Market Overview:**
- The outdoor recreation app market is experiencing robust growth, driven by increased interest in outdoor activities, especially hiking, post-pandemic.
- The global hiking app market is estimated to be valued at over $800 million with a CAGR of 15-20% through 2027.
- Key segments include trail navigation, community features, safety tools, and gamification.

**Key Competitors:**
- **AllTrails:** Market leader with over 30 million users, subscription model at $29.99/year.
- **Gaia GPS:** Strong in navigation features, particularly popular among serious hikers.
- **Hiking Project:** Community-driven content backed by REI.
- **Komoot:** European market leader expanding globally with a focus on route planning.
- **Strava:** While not specific to hik

tool_name=<perplexity_reason> | failed to process tool
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/handlers/tool_handler.py", line 105, in process
    return tool_func.invoke(tool, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/tools/mcp/mcp_agent_tool.py", line 83, in invoke
    return self.mcp_client.call_tool_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 183, in call_tool_sync
    raise MCPClientInitializationError("the client session is not running")
strands.types.exceptions.MCPClientInitializationError: the client session is not running



Tool #6: perplexity_reason
<thinking> It appears that there was an error while attempting to synthesize a comprehensive summary using the reasoning tool. Given the available information and the results from the swarm of specialist agents, I will manually compile a detailed summary based on the collected data.</thinking>


Tool #7: perplexity_research


<thinking> With the research data in hand, I will now create a detailed summary that captures the critical elements of the market analysis.</thinking>

**Comprehensive Summary of Outdoor Recreation App Market Research for a Hiking Trail App Startup:**

**Market Overview:**
- The outdoor recreation app market is experiencing significant growth, driven by increased interest in outdoor activities, particularly hiking, post-pandemic.
- The global hiking app market is estimated to be valued at over $800 million with a Compound Annual Growth Rate (CAGR) of 15-20% through 2027.
- Key segments include trail navigation, community features, safet

tool_name=<perplexity_research> | failed to process tool
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/handlers/tool_handler.py", line 105, in process
    return tool_func.invoke(tool, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/tools/mcp/mcp_agent_tool.py", line 83, in invoke
    return self.mcp_client.call_tool_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 183, in call_tool_sync
    raise MCPClientInitializationError("the client session is not running")
strands.types.exceptions.MCPClientInitializationError: the client session is not running


 to broader features.

This summary encapsulates the key findings from the market research, providing a solid foundation for strategic decision-making in developing a hiking trail app startup.<thinking> It seems that there was an issue with both the reasoning and research tools, preventing us from synthesizing the comprehensive summary automatically. Given this, I will manually construct a detailed summary based on the previously gathered data and insights.</thinking>

**Comprehensive Summary of Outdoor Recreation App Market Research for a Hiking Trail App Startup:**

**Market Overview:**
- The outdoor recreation app market is experiencing significant growth, driven by increased interest in outdoor activities, particularly hiking, post-pandemic.
- The global hiking app market is estimated to be valued at over $800 million with a Compound Annual Growth Rate (CAGR) of 15-20% through 2027.
- Key segments include trail navigation, community features, safety tools, and gamification.

**Key 

---

## Step 4: Specialized Agent Roles

Now we'll create **specialized agents** with distinct personalities and expertise:

- **Market Analyst**: Data-focused, finds and analyzes market information
- **Strategic Advisor**: Business-focused, creates actionable strategies

This demonstrates how different system prompts create agents with unique capabilities and perspectives.

<img src="architecture/step4.png" alt="Description" width="500">


In [13]:
# print_step(4, "Specialized Agent Roles - Market Analyst & Strategic Advisor")

# def create_specialist_team():
#     """Create our specialized agent team"""
    
#     # Market Analyst - specializes in data gathering and analysis
#     market_analyst = Agent(
#         system_prompt="""You are a Senior Market Analyst at a top consulting firm.
#         Your expertise is gathering market data, analyzing competitors, and identifying trends.
#         You provide detailed, data-driven insights with specific numbers and facts.
#         Always support your analysis with concrete data points.""",
#         model=model,
#         tools=([swarm] + (search_client.list_tools_sync() if search_client else []))
#     )
    
#     # Strategic Advisor - specializes in business strategy and recommendations
#     strategic_advisor = Agent(
#         system_prompt="""You are a Strategic Business Advisor with 15+ years of experience.
#         You excel at translating market research into actionable business strategies.
#         Focus on practical recommendations for positioning, pricing, marketing, and growth.
#         Provide clear, implementable advice that startups can act on immediately.""",
#         model=model
#     )
    
#     print("👥 Specialist team created:")
#     print("   Market Analyst - Data gathering and competitive analysis")
#     print("   Strategic Advisor - Business strategy and recommendations")
    
#     return market_analyst, strategic_advisor

# # Create our specialist team
# analyst, strategist = create_specialist_team()


In [14]:
def create_specialist_team():
    """Create our specialized agent team"""
    
    # Get tools within session if available
    with search_client:  # ✅ Session managed inside function
        search_tools = search_client.list_tools_sync()
        analyst_tools = [swarm] + search_tools
    
    
    # Market Analyst - specializes in data gathering and analysis
    market_analyst = Agent(
        system_prompt="""You are a Senior Market Analyst at a top consulting firm.
        Your expertise is gathering market data, analyzing competitors, and identifying trends.
        You provide detailed, data-driven insights with specific numbers and facts.
        Always support your analysis with concrete data points.""",
        model=model,
        tools=analyst_tools  # ✅ Use tools obtained in session
    )
    
    # Strategic Advisor - specializes in business strategy and recommendations
    strategic_advisor = Agent(
        system_prompt="""You are a Strategic Business Advisor with 15+ years of experience.
        You excel at translating market research into actionable business strategies.
        Focus on practical recommendations for positioning, pricing, marketing, and growth.
        Provide clear, implementable advice that startups can act on immediately.""",
        model=model
    )
    
    print("👥 Specialist team created:")
    print("   Market Analyst - Data gathering and competitive analysis")
    print("   Strategic Advisor - Business strategy and recommendations")
    
    return market_analyst, strategic_advisor

# ✅ Simple call - session handled inside
analyst, strategist = create_specialist_team()

👥 Specialist team created:
   Market Analyst - Data gathering and competitive analysis
   Strategic Advisor - Business strategy and recommendations


In [15]:
def run_specialist_workflow(startup_idea):
    """Run a complete workflow using our specialist agents"""
    
    print(f"🎯 Analyzing startup: {startup_idea}")
    
    # PHASE 1: Market Analysis
    print("\nPHASE 1: Market Analysis")
    print("-" * 50)
    
    research_task = f"""
    Analyze the market for this startup idea: {startup_idea}
    
    Research and provide:
    1. Market size and growth potential
    2. Key competitors and their market share
    3. Target customer segments and their needs
    4. Market trends and opportunities
    
    Use available tools to gather current market data.
    """
    
    market_research = analyst(research_task);
    
    print("✅ Market analysis complete!")
    
    # PHASE 2: Strategy Development
    print("\nPHASE 2: Strategy Development")
    print("-" * 50)
    
    strategy_task = f"""
    Based on this market research, develop a business strategy for: {startup_idea}
    
    Market Analysis:
    {market_research}
    
    Provide strategic recommendations for:
    1. Value proposition and positioning
    2. Target customer segments and personas
    3. Competitive differentiation
    4. Go-to-market strategy
    5. Revenue model and pricing
    6. Key success metrics to track
    """
    
    business_strategy = strategist(strategy_task);
    
    print("✅ Strategy development complete!")
    
    return market_research, business_strategy

# Test our specialist workflow
test_startup = "A mobile app that uses AR to help people learn cooking skills through interactive recipes"

research_results, strategy_results = run_specialist_workflow(test_startup)

# Display results
print_response(str(research_results)[:500] + "...", "Market Research Summary (Truncated)")
print_response(str(strategy_results)[:500] + "...", "Strategic Recommendations Summary (Truncated)")


🎯 Analyzing startup: A mobile app that uses AR to help people learn cooking skills through interactive recipes

PHASE 1: Market Analysis
--------------------------------------------------
<thinking> To analyze the market for the startup idea of a mobile app that uses AR to help people learn cooking skills through interactive recipes, I need to gather and synthesize data from various sources. Here's the plan:

1. Use `perplexity_research` to gather data on the market size and growth potential for cooking education apps, specifically those using AR technology.
2. Again, use `perplexity_research` to identify key competitors in the AR cooking education app market and their respective market shares.
3. Use `perplexity_research` to determine the target customer segments for this type of app and their specific needs.
4. Finally, use `perplexity_research` to find out the current market trends and opportunities in the AR cooking education app sector.

I will then

tool_name=<perplexity_research> | failed to process tool
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/handlers/tool_handler.py", line 105, in process
    return tool_func.invoke(tool, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/tools/mcp/mcp_agent_tool.py", line 83, in invoke
    return self.mcp_client.call_tool_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/aws_p3.12/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 183, in call_tool_sync
    raise MCPClientInitializationError("the client session is not running")
strands.types.exceptions.MCPClientInitializationError: the client session is not running
tool_name=<perplexity_research> | failed to process tool
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/m

 compile this information into a comprehensive analysis for the startup idea.</thinking>


Tool #8: perplexity_research

Tool #9: perplexity_research

Tool #10: perplexity_research

Tool #11: perplexity_research
<thinking> It seems that the `perplexity_research` tool is currently not operational, which is preventing me from gathering the necessary market data. Without this data, I cannot provide the full analysis requested for the startup idea.

However, I can still provide some general insights based on what is known about the market for cooking education apps and AR technology. I can also suggest alternative sources or methods for obtaining the required data if the tools become available again. If the user agrees, I can proceed with this approach or wait for the tools to become operational.</thinking>



<thinking> Given the current situation, I'll inform the user about the tools' unavailability and offer to wait for them to become operational or suggest alternative methods for data 

---

## Step 5: Content Creation and File Operations

Our final step adds **content creation and file management** capabilities:

- **Content Writer**: Creates marketing materials and campaigns
- **Report Writer**: Generates professional documentation and saves files

<img src="architecture/step5.png" alt="Description" width="500">


In [19]:
print_step(5, "Content Creation and File Operations")

# Import file operation tools
from strands_tools import file_write, editor


os.environ['BYPASS_TOOL_CONSENT'] = "true"

def create_content_team():
    """Create agents that can generate content and save files"""
    
    # Content Writer - creates engaging marketing materials
    content_writer = Agent(
        system_prompt="""You are a Creative Marketing Writer specializing in startup marketing.
        You excel at creating compelling copy, campaign ideas, and promotional content.
        Transform business strategies into engaging narratives that resonate with target audiences.
        Focus on benefits, emotional appeal, and clear calls to action.""",
        model=model
    )
    
    # Report Writer - creates professional documentation
    report_writer = Agent(
        system_prompt="""You are a Professional Report Writer who creates executive-level documents.
        Take research and strategy outputs and format them into clean, professional reports.
        Use proper markdown structure, headings, and formatting for easy reading.
        Always save your reports using the file_write toolto save reports.""",
        model=model,
        tools=[file_write, editor]  # This agent can create and edit files!
    )
    
    print("👥 Content team created:")
    print("   Content Writer - Marketing materials and campaigns")
    print("   Report Writer - Professional documentation")
    
    return content_writer, report_writer

# Create our content team
writer, reporter = create_content_team()



🎯 STEP 5: Content Creation and File Operations
👥 Content team created:
   Content Writer - Marketing materials and campaigns
   Report Writer - Professional documentation


In [20]:
def test_content_creation():
    """Test content creation and file operations"""
    
    # Sample business strategy (normally this comes from previous steps)
    sample_strategy = """
    Business Strategy for AR Cooking Learning App:
    
    • Target Market: Millennials and Gen Z who want to learn cooking (ages 22-40)
    • Value Proposition: "Learn cooking like having a chef in your kitchen"
    • Key Features: AR recipe overlays, real-time guidance, ingredient recognition
    • Revenue Model: Freemium with premium recipes and advanced features
    • Go-to-Market: Social media, cooking influencers, kitchen appliance partnerships
    """
    
    print("Creating marketing content...")
    
    # STEP 1: Content Writer creates marketing materials
    content_task = f"""
    Create compelling marketing content based on this strategy:
    {sample_strategy}
    
    Generate:
    1. A catchy tagline and value proposition
    2. Key marketing messages for social media
    3. Brief description for app store listing
    4. Elevator pitch for investors (30 seconds)
    
    Make it engaging and action-oriented.
    """
    
    marketing_content = writer(content_task);
    
    print("✅ Marketing content created!")
    print_response(str(marketing_content)[:400] + "...", "Marketing Materials Preview")
    
    print("\nCreating professional report...")
    
    # STEP 2: Report Writer formats and saves everything
    report_task = f"""
    Create a comprehensive startup advisory report that includes:
    
    Business Strategy:
    {sample_strategy}
    
    Marketing Materials:
    {marketing_content}
    
    Format this as a professional markdown report with:
    - Executive Summary
    - Business Strategy Overview  
    - Marketing Recommendations
    - Next Steps
    
    Save the report as 'startup_advisory_report.md'
    """
    
    report_result = reporter(report_task);
    
    print("✅ Professional report created and saved!")
    print_response(str(report_result), "Report Generation Result")
    
    return marketing_content, report_result

# Test content creation
marketing_materials, report_status = test_content_creation()



Creating marketing content...
### 1. Catchy Tagline and Value Proposition

**Tagline:** "Cook Like a Pro, Anytime, Anywhere!"

**Value Proposition:** "Unlock the secret to mastering the kitchen with our AR Cooking Learning App. Imagine having a personal chef right in your kitchen—learn cooking like having a chef in your kitchen, step-by-step, with our cutting-edge AR technology."

### 2. Key Marketing Messages for Social Media

**Instagram:**
- "🌟 Ready to become a cooking pro? Our AR Cooking Learning App turns your kitchen into a classroom! 🌟 #CookLikeAPro #ARCooking #KitchenMagic"
- "👩‍🍳👨‍🍳 Tired of cooking disasters? Let our AR overlays guide you through every recipe! 🌮🥗 #LearnToCook #ARChef #KitchenHacks"

**Facebook:**
- "🔍 Discover the future of cooking with our AR Cooking Learning App! Get real-time guidance and see recipes come to life in your kitchen. 🌟 #CookWithAR #FutureOfCooking #KitchenInnovators"
- "🌟 Ready to impress your friends and family with delicious meals? Join the

╔══════ File Write Operation ══════╗
║                                  ║
║ Path: startup_advisory_report.md ║
║ Size: 3663 characters            ║
║                                  ║
╚══════════════════════════════════╝

╔═══════════════════ Write Successful ════════════════════╗
║ File written successfully to startup_advisory_report.md ║
╚═════════════════════════════════════════════════════════╝

The startup advisory report has been successfully created and saved as 'startup_advisory_report.md'. This report includes an executive summary, a detailed business strategy overview, comprehensive marketing recommendations, and outlined next steps for the AR Cooking Learning App.

You can access the report by looking for the file named 'startup_advisory_report.md' in the appropriate directory on your system. If you need any further assistance or have additional requests, feel free to let me know!✅ Professional report created and saved!

🤖 Report Generation Result
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
The startup advisory report has been successfully created and saved as 'startup_advisory_report.md'. This report
includes an executive summary, a detailed business strategy overview, comprehensive marketing recommendations, and
outlined next steps for the AR Cooking Learning App.  You can access the report b

---

## Complete System Demo

Now let's put it all together! This function demonstrates the complete startup advisor system working end-to-end:

1. **Market Research** → Data gathering and analysis
2. **Strategy Development** → Business planning and recommendations  
3. **Content Creation** → Marketing materials and campaigns
4. **Report Generation** → Professional documentation

<img src="architecture/step6.png" alt="Description" width="1000">


In [24]:
import os
from mcp import StdioServerParameters, stdio_client
from strands import Agent
from strands.tools.mcp import MCPClient
from strands_tools import file_write, editor

def complete_startup_advisory(startup_description):
    """Complete end-to-end startup advisory workflow"""
    
    print("=" * 60)
    print(f"💡 Startup Idea: {startup_description}")
    
    results = {}
    
    # Configure MCP client for Perplexity search
    PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")
    search_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="docker",
                args=[
                    "run",
                    "-i",
                    "--rm",
                    "-e",
                    "PERPLEXITY_API_KEY",
                    "mcp/perplexity-ask",
                ],
                env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY},
            )
        )
    )
    
    # PHASE 1: Market Research  
    print("\nPHASE 1: Market Research & Analysis")
    print("-" * 50)
    
    research_task = f"""
    Conduct comprehensive market research for: {startup_description}
    
    Research:
    1. Market size and growth potential
    2. Key competitors and their positioning
    3. Target customer segments
    4. Market opportunities and challenges
    
    Be comprehensive and data-driven in your analysis.
    """
    
    with search_client:
        search_tools = search_client.list_tools_sync()
        analyst = Agent(
            system_prompt="""You are a senior market research analyst with expertise in startup ecosystems. 
            Use web search to gather current market data and competitive intelligence.""",
            tools=[swarm] + search_tools
        )
        market_research = analyst(research_task)
    
    results['research'] = market_research
    print("✅ Market research completed")
    
    # PHASE 2: Strategy Development
    print("\nPHASE 2: Business Strategy Development")
    print("-" * 50)
    
    strategy_task = f"""
    Develop a comprehensive business strategy based on this research:
    
    Startup: {startup_description}
    Research: {market_research}
    
    Provide detailed recommendations for:
    1. Value proposition and market positioning
    2. Target customer segments and personas
    3. Competitive differentiation strategy
    4. Go-to-market plan and channels
    5. Revenue model and pricing strategy
    6. Key success metrics and milestones
    """
    
    with search_client:
        search_tools = search_client.list_tools_sync()
        strategist = Agent(
            system_prompt="""You are a business strategy consultant specializing in startup growth. 
            Develop comprehensive strategies based on market research.""",
            tools=search_tools
        )
        business_strategy = strategist(strategy_task)
    
    results['strategy'] = business_strategy
    print("✅ Business strategy developed")
    
    # PHASE 3: Marketing Content
    print("\nPHASE 3: Marketing Content Creation")
    print("-" * 50)
    
    content_task = f"""
    Create compelling marketing materials for: {startup_description}
    
    Based on strategy: {business_strategy}
    
    Develop:
    1. Brand positioning and tagline
    2. Key value propositions for different audiences
    3. Social media campaign concepts
    4. Investor pitch summary
    5. App store description
    
    Make it engaging and actionable.
    """
    
    writer = Agent(
        system_prompt="""You are a marketing content specialist who creates compelling 
        campaigns and messaging for startups."""
    )
    marketing_content = writer(content_task)
    
    results['marketing'] = marketing_content
    print("✅ Marketing content created")
    
    # PHASE 4: Professional Report
    print("\nPHASE 4: Professional Report Generation")
    print("-" * 50)
    
    report_task = f"""
    Create a comprehensive startup advisory report including all our analysis:
    
    Startup: {startup_description}
    Market Research: {market_research}
    Business Strategy: {business_strategy}
    Marketing Content: {marketing_content}
    
    Format as a professional executive report with:
    - Executive Summary
    - Market Analysis
    - Strategic Recommendations
    - Marketing Plan
    - Implementation Roadmap
    - Appendices
    
    Save as 'complete_startup_advisory_report.md'
    """
    
    reporter = Agent(
        system_prompt="""You are a professional report writer who creates comprehensive 
        business reports in markdown format. Save reports using the file_write tool.""",
        tools=[file_write, editor]
    )
    final_report = reporter(report_task)
    
    results['report'] = final_report
    print("✅ Professional report generated and saved")
    
    # Summary
    print("\nSTARTUP ADVISORY COMPLETE!")
    print("=" * 60)
    # print("📋 Deliverables generated:")
    # print("   Market research and competitive analysis")
    # print("   Business strategy and recommendations")
    # print("   Marketing content and campaigns")
    # print("   Professional advisory report (saved to file)")
    
    return results


In [ ]:
# def complete_startup_advisory(startup_description):
#     """Complete end-to-end startup advisory workflow"""
    
#     print("=" * 60)
#     print(f"💡 Startup Idea: {startup_description}")
    
#     results = {}
    
#     # PHASE 1: Market Research  
#     print("\nPHASE 1: Market Research & Analysis")
#     print("-" * 50)
    
#     research_task = f"""
#     Conduct comprehensive market research for: {startup_description}
    
#     Research:
#     1. Market size and growth potential
#     2. Key competitors and their positioning
#     3. Target customer segments
#     4. Market opportunities and challenges
    
#     Be comprehensive and data-driven in your analysis.
#     """
    
#     with search_client:
#         market_research = analyst(research_task);
    
    
#     results['research'] = market_research
#     print("✅ Market research completed")
    
#     # PHASE 2: Strategy Development
#     print("\nPHASE 2: Business Strategy Development")
#     print("-" * 50)
    
#     strategy_task = f"""
#     Develop a comprehensive business strategy based on this research:
    
#     Startup: {startup_description}
#     Research: {market_research}
    
#     Provide detailed recommendations for:
#     1. Value proposition and market positioning
#     2. Target customer segments and personas
#     3. Competitive differentiation strategy
#     4. Go-to-market plan and channels
#     5. Revenue model and pricing strategy
#     6. Key success metrics and milestones
#     """
    
#     business_strategy = strategist(strategy_task);
#     results['strategy'] = business_strategy
#     print("✅ Business strategy developed")
    
#     # PHASE 3: Marketing Content
#     print("\nPHASE 3: Marketing Content Creation")
#     print("-" * 50)
    
#     content_task = f"""
#     Create compelling marketing materials for: {startup_description}
    
#     Based on strategy: {business_strategy}
    
#     Develop:
#     1. Brand positioning and tagline
#     2. Key value propositions for different audiences
#     3. Social media campaign concepts
#     4. Investor pitch summary
#     5. App store description
    
#     Make it engaging and actionable.
#     """
    
#     marketing_content = writer(content_task);
#     results['marketing'] = marketing_content
#     print("✅ Marketing content created")
    
#     # PHASE 4: Professional Report
#     print("\nPHASE 4: Professional Report Generation")
#     print("-" * 50)
    
#     report_task = f"""
#     Create a comprehensive startup advisory report including all our analysis:
    
#     Startup: {startup_description}
#     Market Research: {market_research}
#     Business Strategy: {business_strategy}
#     Marketing Content: {marketing_content}
    
#     Format as a professional executive report with:
#     - Executive Summary
#     - Market Analysis
#     - Strategic Recommendations
#     - Marketing Plan
#     - Implementation Roadmap
#     - Appendices
    
#     Save as 'complete_startup_advisory_report.md'
#     """
    
#     final_report = reporter(report_task);
#     results['report'] = final_report
#     print("✅ Professional report generated and saved")
    
#     # Summary
#     print("\nSTARTUP ADVISORY COMPLETE!")
#     print("=" * 60)
#     print("📋 Deliverables generated:")
#     print("   Market research and competitive analysis")
#     print("   Business strategy and recommendations")
#     print("   Marketing content and campaigns")
#     print("   Professional advisory report (saved to file)")
    
#     return results


In [ ]:
# ... existing code ...
def complete_startup_advisory(startup_description):
    """
    Run the full pipeline:
        1. Market research   (Analyst + Perplexity MCP tools)
        2. Business strategy (Strategist + Perplexity MCP tools)
        3. Marketing content (Writer – pure LLM)
        4. Exec report       (Reporter – saves markdown file)

    Returns a dict with the four phase results.
    """

    print("=" * 60)
    print(f"💡 Startup Idea: {startup_description}")

    results = {}

    # ------------------------------------------------------------------ #
    # 0️⃣  MCP client – Perplexity search container
    # ------------------------------------------------------------------ #
    PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")
    if not PERPLEXITY_API_KEY:
        raise ValueError("⛔  Please set PERPLEXITY_API_KEY in your .env file")

    search_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="docker",                # use docker/finch depending on OS
                args=[
                    "run", "-i", "--rm",
                    "-e", "PERPLEXITY_API_KEY",
                    "mcp/perplexity-ask",
                ],
                env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY},
            )
        )
    )

    # ------------------------------------------------------------------ #
    # 1️⃣  PHASE 1 – Market research
    # ------------------------------------------------------------------ #
    print("\nPHASE 1: Market Research & Analysis")
    print("-" * 50)

    research_task = f"""
    Conduct comprehensive market research for: {startup_description}

    Research:
    1. Market size and growth potential
    2. Key competitors and their positioning
    3. Target customer segments
    4. Market opportunities and challenges

    Be comprehensive and data-driven in your analysis.
    """

    with search_client:                                   # session must be active
        perplexity_tools = search_client.list_tools_sync()
        analyst = Agent(
            system_prompt="""You are a senior market-research analyst.
            Use web search to gather current market data and competitive intelligence.""",
            tools=[swarm] + perplexity_tools              # swarm + search
        )
        market_research = analyst(research_task)

    results["research"] = market_research
    print("✅ Market research completed")

    # ------------------------------------------------------------------ #
    # 2️⃣  PHASE 2 – Business strategy
    # ------------------------------------------------------------------ #
    print("\nPHASE 2: Business Strategy Development")
    print("-" * 50)

    strategy_task = f"""
    Develop a comprehensive business strategy based on this research:

    Startup: {startup_description}
    Research: {market_research}

    Provide detailed recommendations for:
    1. Value proposition and market positioning
    2. Target customer segments and personas
    3. Competitive differentiation strategy
    4. Go-to-market plan and channels
    5. Revenue model and pricing strategy
    6. Key success metrics and milestones
    """

    with search_client:
        perplexity_tools = search_client.list_tools_sync()
        strategist = Agent(
            system_prompt="""You are a business-strategy consultant specialising in startup growth.
            Develop comprehensive, actionable strategies based on market research.""",
            tools=perplexity_tools                       # only search tools needed
        )
        business_strategy = strategist(strategy_task)

    results["strategy"] = business_strategy
    print("✅ Business strategy developed")

    # ------------------------------------------------------------------ #
    # 3️⃣  PHASE 3 – Marketing content (no external tools)
    # ------------------------------------------------------------------ #
    print("\nPHASE 3: Marketing Content Creation")
    print("-" * 50)

    content_task = f"""
    Create compelling marketing materials for: {startup_description}

    Based on strategy: {business_strategy}

    Develop:
    1. Brand positioning and tagline
    2. Key value propositions for different audiences
    3. Social-media campaign concepts
    4. Investor pitch summary
    5. App-store description

    Make it engaging and actionable.
    """

    writer = Agent(
        system_prompt="""You are a marketing-content specialist who creates
        compelling campaigns and messaging for startups."""
    )
    marketing_content = writer(content_task)

    results["marketing"] = marketing_content
    print("✅ Marketing content created")

    # ------------------------------------------------------------------ #
    # 4️⃣  PHASE 4 – Executive report (uses file_write)
    # ------------------------------------------------------------------ #
    print("\nPHASE 4: Professional Report Generation")
    print("-" * 50)

    report_task = f"""
    Create a comprehensive startup advisory report including all our analysis:

    Startup: {startup_description}
    Market Research: {market_research}
    Business Strategy: {business_strategy}
    Marketing Content: {marketing_content}

    Format as a professional markdown report with:
    - Executive Summary
    - Market Analysis
    - Strategic Recommendations
    - Marketing Plan
    - Implementation Roadmap
    - Appendices

    Save as 'complete_startup_advisory_report.md'
    """

    reporter = Agent(
        system_prompt="""You are a professional report writer who produces
        executive-level markdown reports. Save reports using the file_write tool.""",
        tools=[file_write, editor]
    )
    final_report = reporter(report_task)

    results["report"] = final_report
    print("✅ Professional report generated and saved")

    # ------------------------------------------------------------------ #
    # ✔️  Wrap-up
    # ------------------------------------------------------------------ #
    print("\nSTARTUP ADVISORY COMPLETE!")
    print("=" * 60)
    return results

In [ ]:
# Test the complete system with a sample startup
startup_idea = "A mobile app that helps people find and share local hiking trails"


print(f"Testing complete system with: {startup_idea}")
print("\nThis will take a few moments as each agent does their work...")

# Run the complete workflow
all_results = complete_startup_advisory(startup_idea)

# Display preview of results
print("\nResults Preview:")
print("=" * 50)

for phase, result in all_results.items():
    preview = str(result)[:200] + "..." if len(str(result)) > 200 else str(result)
    print(f"\n{phase.upper()}: {preview}")



Testing complete system with: A mobile app that helps people find and share local hiking trails

This will take a few moments as each agent does their work...
💡 Startup Idea: A mobile app that helps people find and share local hiking trails

PHASE 1: Market Research & Analysis
--------------------------------------------------
I

##  Demo

Now you can test the complete system with your own startup ideas! Try different types of businesses to see how the agents adapt their analysis and recommendations.


In [22]:
# Test the complete system with a sample startup
startup_idea = "A mobile app that helps people find and share local hiking trails"


print(f"Testing complete system with: {startup_idea}")
print("\nThis will take a few moments as each agent does their work...")

# Run the complete workflow
all_results = complete_startup_advisory(startup_idea)

# Display preview of results
print("\nResults Preview:")
print("=" * 50)

for phase, result in all_results.items():
    preview = str(result)[:200] + "..." if len(str(result)) > 200 else str(result)
    print(f"\n{phase.upper()}: {preview}")



Testing complete system with: A subscription service that delivers personalized vitamin packages based on DNA analysis and lifestyle data

This will take a few moments as each agent does their work...
💡 Startup Idea: A subscription service that delivers personalized vitamin packages based on DNA analysis and lifestyle data

PHASE 1: Market Research & Analysis
--------------------------------------------------
<thinking> To conduct comprehensive market research for a subscription service that delivers personalized vitamin packages based on DNA analysis and lifestyle data, I need to gather and synthesize data from various sources. Here's the plan:

1. Use `perplexity_research` to gather data on the market size and growth potential for personalized vitamin subscription services.
2. Again, use `perplexity_research` to identify key competitors in the personalized vitamin subscription market and their positioning.
3. Use `perplexity_research` to determine the target customer segments for thi

╔══════════ File Write Operation ═══════════╗
║                                           ║
║ Path: complete_startup_advisory_report.md ║
║ Size: 3384 characters                     ║
║                                           ║
╚═══════════════════════════════════════════╝

╔════════════════════════ Write Successful ════════════════════════╗
║ File written successfully to complete_startup_advisory_report.md ║
╚══════════════════════════════════════════════════════════════════╝

The comprehensive startup advisory report for the personalized vitamin subscription service has been successfully created and saved as 'complete_startup_advisory_report.md'. This report includes an executive summary, detailed market analysis, strategic recommendations, a marketing plan, an implementation roadmap, and appendices. It's designed to guide the startup's strategy and growth in the rapidly expanding market for personalized vitamin subscriptions.

You can access the report by looking for the file named 'complete_startup_advisory_report.md' in the appropriate directory on your system. If you need any further assistance or have additional requests, feel free to let me know!✅ Professional report generated and saved

STARTUP ADVISORY COMPLETE!
📋 Deliverables generated:
   Market research and competitive analysis
   Business strategy and recommendations
   Marketing content and campaigns
   Professional advisory report (saved to file)

Results Preview:

RESEARCH: The comprehensive 

In [ ]:
# Try with your own startup idea!
# Uncomment and modify this section to test your own ideas

# your_startup_idea = "Enter your startup idea here..."
# 
# print(f"🚀 Analyzing your startup: {your_startup_idea}")
# your_results = complete_startup_advisory(your_startup_idea)
# 
# print("\n🎉 Your startup analysis is complete!")
# print("📄 Check the generated report for detailed recommendations.")

print("💡 Ready to analyze your startup idea!")
print("✏️ Uncomment the code above and add your startup description to get started.")
